In [46]:
from pymorphy2 import MorphAnalyzer
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('dataset.csv')
df.head()

,uu_usl_name,CLASS_ID,SERVICE_CLASS_CONFIRMED,CNT_DIST_USL_BY_CLASS,CNT_NUM_USL
0,АМБУЛАТОРНАЯ ПОМОЩЬ,1000,NaN,12614,812414
1,АМБУЛАТОРНАЯ ПОМОЩЬ,1292,NaN,777,812414
2,КОНСУЛЬТАТИВНО-ДИАГНОСТИЧЕСКАЯ ПОМОЩЬ,1000,NaN,12614,740580
3,Сбербанк Премьер,3200,1.0,3974,489254
4,Сбербанк Премьер,8350,1.0,104,489254


In [57]:
def prepare_Xy():
    df = pd.read_csv('dataset.csv')
    df = df.query('CLASS_ID != 9000')
    X = df['uu_usl_name']
    y = df['CLASS_ID']
    return X,y
X, y = prepare_Xy()

In [58]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
X = X.apply(lambda x: ' '.join([morph.parse(word)[0].normal_form for word in x.split()]))

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
tX = cv.fit_transform(X)

In [62]:
tX.shape

(81431, 17707)

In [63]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(tX,y,test_size=0.2)


In [64]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [67]:
pred = model.predict(X_test)
print(sum(pred == y_test)/len(y_test))
# pd.Series(pred).value_counts()

0.4717873150365322


In [68]:
from joblib import dump, load
dump(model, 'simple_vectorized_model.joblib')
dump(cv, 'simple_vectorized_cv.joblib')

['simple_vectorized_cv.joblib']

In [69]:
def simple_model_pipeline(X):
    from joblib import load
    model = load('simple_vectorized_model.joblib')
    cv = load('simple_vectorized_cv.joblib')
    pred_proba = model.predict_proba(cv.transform(X))
    classes = model.classes_
    return pd.DataFrame(pred_proba, columns=classes)